## Testing the Python implementation of PDM

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot
import hvplot.pandas
from datetime import timedelta

%load_ext autoreload
%autoreload 2

In [ ]:
pad = Path(os.getcwd())
pad.name

Set Path to the above folder = the main Python folder

In [ ]:
if pad.name != "Python":
    pad_correct = Path("../../Python")
    os.chdir(pad_correct)
from functions.PDM import PDM

In [ ]:
area = pd.read_csv('matlab/data/Area.csv', header= None, dtype = np.float32)
A = np.single(area.values[0][0])
display(area)
display(A)
type(A)

In [ ]:
A

In [ ]:
colnames = ['cmax','cmin','b','be','k1','k2','kb',
            'kg','St','bg','tdly','qconst','rainfac']
parameters = pd.read_csv('matlab/data/paramPDM.csv', names = colnames, header = None,
dtype = np.float32)
parameters

In [ ]:
P = pd.read_csv("matlab/data/P.csv", names = ['P'], dtype = np.float32)
fig, ax = plt.subplots()
ax.plot(P)

In [ ]:
any(np.isnan(P))

In [ ]:
EP = pd.read_csv("matlab/data/EP.csv", names = ['EP'], dtype = np.float32)
fig, ax = plt.subplots()
ax.plot(EP)

In [ ]:
time = pd.date_range(start = '01/01/1971', end = '31/12/2000', freq = 'H')
print(type(time))
time

In [ ]:
time_daily = pd.date_range(start = time[0], end = time[-1], freq ='4D')
time_daily

In [ ]:
observations = pd.read_csv(Path("matlab/output/Qmod.csv"), names = ["Q_obs"])
observations.shape

## Calling the function and comparing with matlab function and observed data

In [ ]:
deltat = 1
deltat_out = 24
time_in = time[:len(P.values)]
#qmod, tmod, Cstar, qbm3s, qsm3s, Eiacc = PDM(P.values, EP.values, time_in, A,deltat, deltat_out, parameters)
pd_out = PDM(P.values, EP.values, time_in, A,deltat, deltat_out, parameters)

In [ ]:
P.values.shape

In [ ]:
pd_out

Loading in matlab output

In [ ]:
qmod_pd = pd.read_csv("matlab/output/Qmod.csv", dtype = np.float32, header = None,names = ['Matlab'])
pd_qmod_obs = pd.read_csv("matlab/data/observations.csv", dtype = np.float32, header = None, names = ['Observed'])
qmod_matlab = qmod_pd['Matlab'].values
qmod_obs = pd_qmod_obs['Observed'].values
qmod_pd['Python'] = pd_out['qmodm3s']
qmod_pd['Observed'] = pd_qmod_obs
qmod_pd['Time'] = pd_out['Time']
display(qmod_pd)

In [ ]:
#fig, ax = plt.subplots(figsize = (10,8))
#ax.plot(tmod, qmod, label = 'Python')
#ax.plot(tmod, qmod_matlab, linestyle = 'dotted', label ='Matlab')
#ax.plot(tmod, qmod_obs, linestyle = 'dashed', label = 'observed')
#ax.legend()
qmod_pd.plot(x = 'Time', y = ['Python','Observed','Matlab'])

In [ ]:
qmod_pd.hvplot.line(x = 'Time', y = ['Matlab','Python','Observed'],height = 500, width = 800)


In [ ]:
fig, ax = plt.subplots(figsize = (6,6))
ax.plot(pd_out['Cstar'])

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(S1)

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(V)

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(qbm3s)

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(S3)

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(qsm3s)

In [ ]:
#fig, ax = plt.subplots(figsize = (6,6))
#ax.plot(di)

### hourly comparison instead of daily

In [ ]:
deltat_out_bis = 1
pd_out_hr = PDM(P.values, EP.values, time_in, A,deltat, deltat_out_bis, parameters)
qmod_hr_matlab = pd.read_csv(Path("matlab/output/Qmod_hr.csv"), dtype = np.float32, header = None)

In [ ]:
pd_out_hr

In [ ]:
print(len(pd_out_hr))
print(len(qmod_hr_matlab))

In [ ]:
#qmod_hr_nonan = qmod_hr[~np.isnan(qmod_hr)]
#tmod_hr_nonan = tmod_hr[~np.isnan(qmod_hr)]

In [ ]:
qmod_hr_matlab = qmod_hr_matlab[:len(pd_out_hr)]

In [ ]:
#concatted = np.concatenate((qmod_hr.reshape((len(qmod_hr),-1)),qmod_hr_matlab.values),axis = 1)
#pd_out_hr= pd.DataFrame(concatted, columns = ['Python','Matlab'])
#pd_out_hr['Time'] = tmod_hr_nonan
pd_out_hr = pd_out_hr.rename(columns = {'qmodm3s':'Python'})
pd_out_hr['Matlab'] = qmod_hr_matlab
pd_out_hr

In [ ]:
pd_out_hr.plot(x= 'Time', y = ['Python','Matlab'])

In [ ]:
pd_out_hr.hvplot(x= 'Time', y = ['Python','Matlab'])

In [ ]:
pd_out_hr['100*(Python-Matlab)'] = (pd_out_hr['Python'] - pd_out_hr['Matlab'])*100

In [ ]:
#pd_out_hr.hvplot(x= 'Time', y = 'Python-Matlab') + pd_out_hr.hvplot(x = 'Time', y = 'Python')
pd_out_hr.hvplot(x= 'Time', y = ['100*(Python-Matlab)','Python'],subplots = True).cols(1)

Clear conclusion: the difference in the implementation when Cstar reaches cmax

### Compare with the adapated matlab version on hourly basis

In [ ]:
qmod_hr_matlab_adapted = pd.read_csv(Path("matlab/output/Qmod_adapted_hr.csv"), dtype = np.float32, header = None)
pd_out_hr['Matlab_adapted'] = qmod_hr_matlab_adapted.values[0:len(pd_out_hr)]
pd_out_hr['100*(Python-Matlab_adapted)'] = (pd_out_hr['Python'] - pd_out_hr['Matlab_adapted'])*100
pd_out_hr.hvplot(x= 'Time', y = ['100*(Python-Matlab_adapted)','Python'],subplots = True).cols(1)

## Mass Balance

All evapotranspiration and precipitation data are expressed in mm/h and are valid for deltat = 1 hour

In [ ]:
total_actual_EP = np.nansum(pd_out_hr['Eiacc'])
total_P = np.sum(P.values)
total_qm3= np.nansum(pd_out_hr['Python'])*3600 #for 1 hour = 3600s
total_qmod = total_qm3*1000/(A*1000**2) #m^3/m^2 and then *1000 for m-> mm
totaltime = totaltime = pd_out_hr['Time'].iloc[-1] - pd_out_hr['Time'].iloc[0]
totaltime_days = totaltime.days + totaltime.seconds/(3600*24)
total_qconstm3 = parameters['qconst'].values*3600*24*totaltime_days
total_qconst = total_qconstm3*1000/(A*1000**2) #to mm
total_net_rain = total_P - total_actual_EP + total_qconst #so account for constant abstraction!!
print(total_net_rain, total_qmod, total_qconst)

In [ ]:
total_qconst

In [ ]:
fig, ax = plt.subplots(figsize = (6,6))
ax.bar(x = ['P','actual_EP','P-actual_EP','Qtot','Qconst'],
height = [total_P, total_actual_EP, total_net_rain[0], total_qmod[0], total_qconst[0]])

In [ ]:
fig, ax = plt.subplots(figsize = (6,6))
ax.bar(x = ['P-actual_EP+Qconst','Qtot'],height = [total_net_rain.item(), total_qmod.item()])

In [ ]:
total_net_rain.item()

In [ ]:
total_net_rain.flatten()